In [10]:
import pandas as pd
import numpy as np

In [6]:
# Read in data from crew csv
crew = pd.read_csv('../data/raw/title.crew.tsv', sep='\t')
ratings = pd.read_csv('../data/raw/title.ratings.tsv', sep='\t')
titles = pd.read_csv('../data/raw/title.basics.tsv', sep='\t')


C:\Users\Joeyz\AppData\Local\Temp\ipykernel_8844\2674795029.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles = pd.read_csv('../data/raw/title.basics.tsv', sep='\t')


In [17]:
titles.replace('\\N', np.nan, inplace=True)
titles['startYear'] = titles['startYear'].astype(float)

In [20]:
titles.tail()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9273127,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273128,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273129,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010.0,NaN,NaN,"Action,Drama,Family"
9273130,tt9916856,short,The Wind,The Wind,0,2015.0,NaN,27,Short
9273131,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014.0,NaN,10,"Adventure,Animation,Comedy"


In [22]:
between_1950_2020_titles = titles[(titles['startYear']>=1950) & (titles['startYear']<=2020) & (titles['titleType'] != 'tvEpisode')]['tconst']

In [23]:
#Heuristic: Let's filter out movies with less than 10000 votes

high_votes = ratings[ratings['numVotes'] > 10000]

In [25]:
votes_and_titles = pd.merge(between_1950_2020_titles, high_votes, on='tconst')

In [ ]:
#Turn the directors column into a list
crew['directors'] = crew['directors'].str.split(',')

In [32]:
# explode directors column
directors = crew[['tconst', 'directors']].explode('directors')


In [34]:
# Merge the directors with the titles
directors_and_titles = pd.merge(directors, votes_and_titles, on='tconst')

In [85]:
from sklearn.preprocessing import normalize

In [94]:
#Create 3 columns with votes, normalized votes, and rankings
director_votes = directors_and_titles.groupby('directors')['numVotes'].sum()
normalized_director_votes = normalize(director_votes.values.reshape(1, -1))
ranked_director_votes = director_votes.rank(ascending=False)
final_director_rankings = pd.DataFrame({'director':director_votes.index, 'rank':ranked_director_votes.values, 'total_votes':director_votes.values, 'normalized_votes': normalized_director_votes[0]})
final_director_rankings.to_csv('../data/interim/director_rankings.csv', index=False)

In [153]:
#combine netflix data with director data

netflix = pd.read_csv('../data/final_data/netflix_final_data.csv')

In [135]:
netflix_crew = pd.merge(netflix, crew, on='tconst', how='inner')

In [138]:
netflix_crew['directors'][0]

['nm1623901']

In [139]:
#make max director rank column

#netflix_crew['directors'] = netflix_crew['directors'].str.split(',')
netflix_crew['max_director_rank'] = netflix_crew['directors'].apply(lambda x: final_director_rankings[final_director_rankings['director'].isin(x)]['rank'].max())

In [154]:
netflix

,tconst,combined_title,release_year_x,is_top10,title,season,is_tv_show
0,tt13353456,#blackAF: Season 1,2020.0,False,#blackaf,1,True
1,tt12759384,(Un)Well: Season 1,2020.0,False,(un)well,1,True
2,tt10803866,10 Days With Santa Claus,2020.0,False,10 days with santa claus,NaN,False
3,tt11823088,100 Humans: Season 1,2020.0,False,100 humans,1,True
4,tt13782052,1000 Miles from Christmas,2021.0,True,1000 miles from christmas,NaN,False
...,...,...,...,...,...,...,...
2222,tt14599438,jeen-yuhs: A Kanye Trilogy,2022.0,True,jeen-yuhs: a kanye trilogy,NaN,False
2223,tt11561206,the goop lab with Gwyneth Paltrow: Season 1,2020.0,False,the goop lab with gwyneth paltrow,1,True
2224,tt8721424,"tick, tick...BOOM!",2021.0,True,"tick, tick...boom!",NaN,False
2225,tt9725830,Òlòtūré,2020.0,False,òlòtūré,NaN,False


In [147]:
directors_and_titles[directors_and_titles['directors']=='nm1623901']
#directors_and_titles

,tconst,directors,averageRating,numVotes


Now Repeat with the Writers

In [95]:
crew['writers'] = crew['writers'].str.split(',')
writers = crew[['tconst', 'writers']].explode('writers')

In [96]:
writers_and_titles = pd.merge(writers, votes_and_titles, on='tconst')

In [97]:

writer_votes = writers_and_titles.groupby('writers')['numVotes'].sum()
normalized_writer_votes = normalize(writer_votes.values.reshape(1, -1))
ranked_writer_votes = writer_votes.rank(ascending=False)
final_writer_rankings = pd.DataFrame({'writer':writer_votes.index, 'rank':ranked_writer_votes.values, 'total_votes':writer_votes.values, 'normalized_votes': normalized_writer_votes[0]})
final_writer_rankings.to_csv('../data/interim/writer_rankings.csv', index=False)

In [100]:
netflix_data = pd.read_csv('../data/final_data/netflix_final_data.csv')

In [104]:
ratings = pd.read_csv('../data/external/title.ratings.tsv', sep='\t')

basics = pd.read_csv('../data/external/title.basics.tsv', sep='\t', usecols=['tconst', 'titleType', 'isAdult', 'runtimeMinutes', 'genres'])

df = netflix_data.merge(ratings, how='left', on='tconst').merge(basics, how='left', on='tconst', )

#df.isna().sum()
df = df[df['tconst'].notna()]
#My Wonderful Life Manually added data

df.loc[df['title']=='my wonderful life', 'averageRating'] = 5.9
df.loc[df['title']=='my wonderful life', 'numVotes'] = 714
df.loc[df['title']=='my wonderful life', 'titleType'] = 'movie'
df.loc[df['title']=='my wonderful life', 'runtimeMinutes'] = 99
df.loc[df['title']=='my wonderful life', 'isAdult'] = 0
df.loc[df['title']=='my wonderful life', 'genres'] = 'Comedy,Drama,Romance'

C:\Users\Joeyz\AppData\Local\Temp\ipykernel_8844\1022789545.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv('../data/external/title.basics.tsv', sep='\t', usecols=['tconst', 'titleType', 'isAdult', 'runtimeMinutes', 'genres'])


In [105]:
df['is_top10'] = df['is_top10'].astype(int)
df.replace('\\N', np.nan, inplace=True)

In [ ]:
movies, tv_shows = df[df['is_tv_show']==False], df[df['is_tv_show']==True]